In [4]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFTGate, grover_operator, MCMTGate, ZGate, HGate
from qiskit_aer import AerSimulator
from qiskit.quantum_info import DensityMatrix
from math import log2, ceil, pi, sqrt
import numpy as np
from enum import Enum
from lib.lib import oracle
import pandas as pd

In [5]:
def encode_number(n: int, d: int) -> str:
    return (format(n, f"0{d}b").replace("-", "1"))[::-1]

def encode_target(T, d) -> str:
    if T < 0:
        return format(2 ** d + T, f"0{d}b")[::-1]
    return format(T, f"0{d}b")[::-1]


def RotationAdd(m, d, old=False):
    n = encode_number(abs(m), d)
    qc = QuantumCircuit(d + 1)
    k = len(n)
    # iterate over bits of n
    angles = np.zeros(d)
    for j in range(k):
        if n[j] == '1':
            for l in range(d - j):
                if not old:
                    angles[l] += np.sign(m) *  2 * pi * 2 ** (l + j) / 2 ** d
                else:
                    qc.cp(np.sign(m) *  2 * pi * 2 ** (l + j) / 2 ** d, 0,1 + l)
    for k in range(k):
        if angles[k] != 0:
            qc.cp(angles[k], 0, 1 + k)
    return qc.to_gate(label=f"R({m})")


In [13]:
def step_by_step_oracle(S, T, step, old=False, uncompute=True):
    s_0 = sum([abs(i) for i in S])
    d = ceil(log2(1 + max(s_0, T))) + (1-(old or T >= 0))

    qc = QuantumCircuit()
    data_register = QuantumRegister(len(S), "index")
    qc.add_register(data_register)
    target_register = QuantumRegister(d, "target")
    data_measure = ClassicalRegister(len(S), "target_M")
    qc.add_register(target_register, data_measure)
    ancilla_register = QuantumRegister(1, "|->")
    #qc.add_register(ancilla_register)

    #qc.x(ancilla_register)
    #qc.h(ancilla_register)
    qc.h(data_register)
    encoded_target = encode_target(T, d)
    for i in range(d):
        if encoded_target[i] == '1':
            qc.x(target_register[i])
    if step == 0:
        qc.save_statevector()

    qc.append(QFTGate(d), target_register)
    if step == 1:
        qc.save_statevector()
    
    for i in range(len(S)):
        n = S[i]
        rotation_adder = RotationAdd(-n, d, old=old)
        qc.append(rotation_adder, [data_register[i]] + list(target_register))
    if step == 2:
        qc.save_statevector()
    qc.append(QFTGate(d).inverse(), target_register)
    if step == 3:
        qc.save_statevector()
    qc.x(target_register)
    if step == 4:
        qc.save_statevector()


    #qc.mcx(target_register, ancilla_register)

    qc.append(MCMTGate(gate=ZGate(), num_ctrl_qubits=d, num_target_qubits=1),[0] + list(range(qc.num_qubits - d, qc.num_qubits)) )
    qc.mcx(target_register, 0)
    qc.append(MCMTGate(gate=ZGate(), num_ctrl_qubits=d, num_target_qubits=1),[0] + list(range(qc.num_qubits - d, qc.num_qubits)))
    qc.mcx(target_register, 0)

    if uncompute:
        qc.x(target_register)
        if step == 5:
            qc.save_statevector()

        qc.append(QFTGate(d), target_register)
    
        for i in range(len(S)):
            n = S[i]
            rotation_adder = RotationAdd(n, d, old=old)
            qc.append(rotation_adder, [data_register[i]] + list(target_register))
                        
        qc.append(QFTGate(d).inverse(), target_register)
        for i in range(d):
            if encoded_target[i] == '1':
                qc.x(target_register[i])
    if step == 6:
        qc.save_statevector()

    return qc


In [15]:
S, T = [1,2], 3
step_mapper = ["target + subset encoded", "QFT", "Rotations", "IQFT", "Target X", "Phase flip", "Uncomputation", "grover"]
for i in range(7):
    qc = step_by_step_oracle(S, T, i)
    simulator = AerSimulator(method='matrix_product_state')
    simulator.set_max_qubits(qc.num_qubits)
    circ = qiskit.transpile(qc, simulator)
    result = simulator.run(circ, shots=4096).result().get_statevector()
    with open(f"{step_mapper[i]}.tex", "w+") as f:
        f.write(result.draw("latex_source", max_size=32))


In [1]:
from lib.lib import test_grovers, simulate
test_grovers([1,2],3, n=1).draw("latex")

<IPython.core.display.Latex object>

In [3]:
import random
import numpy as np
from lib.lib import oracle, simulate
from qiskit_ibm_runtime.fake_provider import *

max_val = 256
n = 10000
#S = np.random.randint(0, max_val, n)
#T = sum(random.choices(S, k=5))
S = [1,2,3,4,5,7,8,9]
T = 5
simulate(S,T, 3, backend=FakeMarrakesh(), noisy=True)

00000110    100
00010000     96
00001001     82
00000001     10
00001011      9
           ... 
01100011      1
11010010      1
01101101      1
01010100      1
11111111      1
Length: 241, dtype: int64

In [2]:
from lib.lib import RotationAdd
from qiskit.circuit.library import QFTGate, grover_operator, MCMTGate, ZGate, HGate

def basis_gate_count_rotation_gate_worst_case(target_size) -> int:
    qc = QuantumCircuit(target_size + 1)
    adder = RotationAdd(2 ** target_size - 1, target_size)
    qc.append(adder, list(range(target_size)))
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).count_ops()

def rotation_gate_worst_case_depth(target_size) -> int:
    qc = QuantumCircuit(target_size + 1)
    adder = RotationAdd(2 ** target_size - 1, target_size)
    qc.append(adder, list(range(target_size + 1)))
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).depth()

def qft_worst_case_gate_count(target_size, inverse=False):
    qc = QuantumCircuit(target_size)
    if inverse:
        qc.append(QFTGate(target_size), list(range(target_size)))
    else:
        qc.append(QFTGate(target_size).inverse(), list(range(target_size)))

    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).count_ops()

def qft_worst_case_depth(target_size, inverse=False):
    qc = QuantumCircuit(target_size)
    if inverse:
        qc.append(QFTGate(target_size), list(range(target_size)))
    else:
        qc.append(QFTGate(target_size).inverse(), list(range(target_size)))
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).depth()

def x_gate_depth():
    qc = QuantumCircuit(1)
    qc.x(0)
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).depth()

def x_gate_gate_count():
    qc = QuantumCircuit(1)
    qc.x(0)
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).count_ops()

def mcmt_depth(target_size):
    qc = QuantumCircuit(target_size)
    qc.append(MCMTGate(gate=ZGate(), num_ctrl_qubits=target_size-1, num_target_qubits=1), list(range(target_size)))
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).depth()

def mcmt_gate_count(target_size):
    qc = QuantumCircuit(target_size)
    qc.append(MCMTGate(gate=ZGate(), num_ctrl_qubits=target_size-1, num_target_qubits=1), list(range(target_size)))
    return qiskit.transpile(qc, basis_gates=["h", "cx", "p"]).depth()


def grover_diffusion_gate_count(data_size):
    qc = QuantumCircuit(data_size)
    grover_op = grover_operator(qc)
    return qiskit.transpile(grover_op, basis_gates=["h", "cx", "p"]).count_ops()

def grover_diffusion_depth(data_size):
    qc = QuantumCircuit(data_size)
    grover_op = grover_operator(qc)
    return qiskit.transpile(grover_op, basis_gates=["h", "cx", "p"]).depth()


In [3]:
import random
import numpy as np
from lib.lib import oracle, simulate
from math import log2, ceil, sqrt

max_val = 256
for n in range(1,10000):
    data_register_size = n
    target_register_size_worst_case = ceil(log2(1 + n * max_val)) + 1
    qbit_count_worst_case = n + target_register_size_worst_case
    depth_worst_case = x_gate_depth()
    depth_worst_case += qft_worst_case_depth(target_register_size_worst_case)
    depth_worst_case += rotation_gate_worst_case_depth(target_register_size_worst_case) * data_register_size
    depth_worst_case += qft_worst_case_depth(target_register_size_worst_case, inverse=True)
    depth_worst_case += x_gate_depth()
    depth_worst_case *= 2 
    depth_worst_case += mcmt_depth(target_register_size_worst_case)
    depth_worst_case += grover_diffusion_depth(data_register_size)
    #depth_worst_case *= ceil(sqrt(2 ** data_register_size / 1) * 0.5)
    display(f"Qbits: {qbit_count_worst_case}, \t  Depth worst case: {depth_worst_case}")

NameError: name 'QuantumCircuit' is not defined

In [ ]:
from lib.lib import groversearch
import pandas as pd

df = pd.DataFrame(columns=["qubits", "depth", "single_gate_count", "multi_gate_count"])

for n in range(1, 10000):
    orcl = groversearch([255] * n, 255*n, 1)
    orcl_t = qiskit.transpile(orcl, basis_gates=["h", "cx", "p"])
    gate_counts = orcl_t.count_ops()
    single_gate_count = gate_counts["h"] + gate_counts["p"]
    multi_gate_count = gate_counts["cx"]
    df.loc[n - 1] = [orcl_t.num_qubits, orcl_t.depth(), single_gate_count, multi_gate_count]
    display(f"QUbits {orcl_t.num_qubits}  Depth: {orcl_t.depth()}, Single gates: {single_gate_count} Multi gates: {multi_gate_count}")

df.to_csv("./scalibility.csv")

'QUbits 9  Depth: 363, Single gates: 615 Multi gates: 320'

'QUbits 11  Depth: 758, Single gates: 1514 Multi gates: 866'

'QUbits 13  Depth: 844, Single gates: 1868 Multi gates: 1136'

'QUbits 14  Depth: 886, Single gates: 2012 Multi gates: 1228'

'QUbits 16  Depth: 1030, Single gates: 2497 Multi gates: 1552'

...

'QUbits 206  Depth: 44706, Single gates: 249089 Multi gates: 166104'

'QUbits 207  Depth: 44882, Single gates: 250353 Multi gates: 166918'

'QUbits 208  Depth: 44970, Single gates: 251560 Multi gates: 167732'

'QUbits 209  Depth: 45235, Single gates: 252818 Multi gates: 168546'

'QUbits 210  Depth: 45411, Single gates: 254025 Multi gates: 169360'

In [ ]:
oracle([1,2,3],1).draw(fold=-1)